In [9]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import os
from tqdm import tqdm
from PIL import Image
from scipy.spatial import distance
from pytorch_fid import fid_score
from prdc import compute_prdc
#noisetype = 'purplestrong'
# Define paths
def calculate_scores(noisetype):
    real_images_dir = "data/cat_res64/0"  # Directory containing real images
    generated_images_dir = f"results_gaussianBN/cat_res64_gaussianBN_sigmoid_1000.0_0_3_outc6_seed0/{noisetype}/images/model_bluev21_750" 

    # Load Inception-v3 model (pretrained on ImageNet)
    inception = models.inception_v3(pretrained=True, transform_input=False).cuda()
    inception.fc = torch.nn.Identity()  # Remove classification layer to get feature vectors
    inception.eval()  # Set to evaluation mode

    # Transformation to match Inception-v3 input requirements
    transform = transforms.Compose([
        transforms.Resize((299, 299)),  # Inception requires 299x299 input
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalization
    ])

    # Function to load images and extract features
    def extract_features(image_dir, model):
        features = []
        image_paths = sorted([os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(".jpg") or img.endswith(".png")])
        
        for img_path in tqdm(image_paths, desc=f"Processing {image_dir}"):
            image = Image.open(img_path).convert("RGB")  # Load and convert to RGB
            image = (transform(image).unsqueeze(0)).cuda()  # Apply transformation
            with torch.no_grad():
                feature = model(image)  # Extract feature vector
            features.append(feature.squeeze().cpu().numpy())  # Convert to numpy
        return np.array(features)

    # Extract feature embeddings
    # real_features = extract_features(real_images_dir, inception)
    # np.save("real_features.npy",real_features)
    fid = fid_score.calculate_fid_given_paths([real_images_dir, generated_images_dir], batch_size=64, device='cuda', dims=2048)
    real_features = np.load("real_features.npy")
    generated_features = extract_features(generated_images_dir, inception)

    # Compute FID (Fréchet Inception Distance)
    
    #print(f"FID Score: {fid_score:.4f}")

    # Compute Precision and Recall
    prdc_metrics = compute_prdc(real_features, generated_features, nearest_k=5)
    prdc_metrics['fid'] = fid
    return prdc_metrics
    # print(f"Precision: {prdc_metrics['precision']:.4f}")
    # print(f"Recall: {prdc_metrics['recall']:.4f}")
    # print(f"Density: {prdc_metrics['density']:.4f}")
    # print(f"Coverage: {prdc_metrics['coverage']:.4f}")

In [10]:
#noisetypes = {"baseline":None,"bluev14":None,"bluev15":None,"bluev21":None,"bluev22":None,"bluev23":None,"bluev24":None,"bluev25":None,"red":None,"white":None,"greenv1":None,"greenv2":None,"purplev1":None,"purplev2":None,"purplev3":None}
noisetypes = {"bluev21":None}
for noisetype in noisetypes:
    noisetypes[noisetype] = calculate_scores(noisetype=noisetype)
    print(f"---metrics for {noisetype}---\nFID: {noisetypes[noisetype]['fid']:.4f}\nPrecision: {noisetypes[noisetype]['precision']:.4f}\nRecall: {noisetypes[noisetype]['recall']:.4f}\nDensity: {noisetypes[noisetype]['density']:.4f}\nCoverage: {noisetypes[noisetype]['coverage']:.4f}\n")
# ---metrics for untrained---
# FID: 8.9628
# Precision: 0.8151
# Recall: 0.6061
# Density: 1.1147
# Coverage: 0.8700

c:\Users\guyle\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\guyle\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 81/81 [00:09<00:00,  8.33it/s]
Processing results_gaussianBN/cat_res64_gaussianBN_sigmoid_1000.0_0_3_outc6_seed0/bluev21/images/model_bluev21_750: 100%|██████████| 5153/5153 [01:37<00:00, 53.04it/s]


Num real: 5153 Num fake: 5153
---metrics for bluev21---
FID: 6.9013
Precision: 0.8494
Recall: 0.6006
Density: 1.2833
Coverage: 0.9156

